In [1]:
# CODE GROUPÉ
# ──────────────────────────────────────────────────────────────────────────────
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os, math, cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# =========================
# PARAMS
# =========================
HFOV_DEG = 93.9
D = np.array([[0.025],[0.045],[0.0],[0.0]], dtype=np.float32)

IMG_PATH   = r"D:\Users\Hp\Desktop\Inspection_Plaques\brute04.jpg"
CSV_PATH   = r"D:\Users\Hp\Desktop\Inspection_Plaques\echelles_bandes.csv"
MODEL_PATH = r"D:\Users\Hp\Desktop\Inspection_Plaques\best (9).pt"
OUT_PATH   = r"D:\Users\Hp\Desktop\Inspection_Plaques\brute04_result.png"

rotation_angle = -3.0  # °

# YOLO + drawing style
CONF, IOU   = 0.30, 0.60
N_LEN, N_WID = 5, 1
COL_V, COL_H = (90,0,90), (90,0,0)
THICK_V, THICK_H = 4,4
MASK_ALPHA = 0.25
FONT = cv2.FONT_HERSHEY_SIMPLEX
TXT_COL, TXT_TH = (255,255,255), 2

# =========================
# 1) READ + UNDISTORT
# =========================
img = cv2.imread(IMG_PATH)
if img is None:
    raise RuntimeError(f"Impossible de lire l'image : {IMG_PATH}")
h, w = img.shape[:2]

f = w / (2 * math.tan(math.radians(HFOV_DEG / 2)))
K = np.array([[f, 0, w/2],
              [0, f, h/2],
              [0, 0, 1]], dtype=np.float32)

new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(
    K, D, (w,h), np.eye(3), balance=1.0
)
map1, map2 = cv2.fisheye.initUndistortRectifyMap(
    K, D, np.eye(3), new_K, (w,h), cv2.CV_16SC2
)
img = cv2.remap(img, map1, map2,
                interpolation=cv2.INTER_LINEAR,
                borderMode=cv2.BORDER_CONSTANT)
print("✅ Undistort appliqué")

# =========================
# 2) ROTATION
# =========================
def rotate_image_keep_bounds(image, angle_deg):
    (h, w) = image.shape[:2]
    cX, cY = w/2, h/2
    M = cv2.getRotationMatrix2D((cX, cY), angle_deg, 1.0)
    cos, sin = abs(M[0,0]), abs(M[0,1])
    new_w, new_h = int(h*sin + w*cos), int(h*cos + w*sin)
    M[0,2] += (new_w/2) - cX
    M[1,2] += (new_h/2) - cY
    return cv2.warpAffine(image, M, (new_w,new_h),
                          flags=cv2.INTER_LINEAR,
                          borderMode=cv2.BORDER_CONSTANT)

img = rotate_image_keep_bounds(img, rotation_angle)
print("✅ Rotation appliquée")

# =========================
# 3) MESURES (identiques à ta version qui marche)
# =========================
# --- CSV des bandes
tab      = pd.read_csv(CSV_PATH)
x_start  = tab["x_start"].to_numpy()
x_end    = tab["x_end"].to_numpy()
px_cm_X  = tab["px_cm_X"].to_numpy()
px_cm_Y  = tab["px_cm_Y"].to_numpy()

def bande_de_x(x: int) -> int:
    for i in range(len(x_start)):
        if x_start[i] <= x < x_end[i]:
            return i
    return 0 if x < x_start[0] else len(x_start)-1

def long_cm_bandes(x0:int, x1:int, x_start, x_end, px_cm_X)->float:
    cm=0.0
    for xs,xe,r in zip(x_start,x_end,px_cm_X):
        left,right=max(x0,xs),min(x1,xe)
        if right>left:
            cm += (right-left)/r
    return cm

def clean_mask(mask: np.ndarray)->np.uint8:
    m = (mask>0).astype(np.uint8)*255
    n,lab,stats,_ = cv2.connectedComponentsWithStats(m)
    if n>1:
        idx = 1+np.argmax(stats[1:,cv2.CC_STAT_AREA])
        m = (lab==idx).astype(np.uint8)*255
    k = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    m = cv2.morphologyEx(m,cv2.MORPH_CLOSE,k,iterations=2)
    m = cv2.morphologyEx(m,cv2.MORPH_OPEN,k,iterations=1)
    return m

def fit_top_bottom_lines(mask: np.ndarray):
    xs = np.where(mask.any(axis=0))[0]
    top_pts, bot_pts=[],[]
    for x in xs:
        ys = np.where(mask[:,x])[0]
        if ys.size:
            top_pts.append([x,ys.min()])
            bot_pts.append([x,ys.max()])
    top_pts, bot_pts = np.array(top_pts,np.float32), np.array(bot_pts,np.float32)
    vxT,vyT,x0T,y0T = cv2.fitLine(top_pts,cv2.DIST_L2,0,0.01,0.01)
    vxB,vyB,x0B,y0B = cv2.fitLine(bot_pts,cv2.DIST_L2,0,0.01,0.01)
    vxT,vyT,x0T,y0T = vxT.item(),vyT.item(),x0T.item(),y0T.item()
    vxB,vyB,x0B,y0B = vxB.item(),vyB.item(),x0B.item(),y0B.item()
    def y_top_at(x): return int(round(y0T+(vyT/vxT)*(x-x0T)))
    def y_bot_at(x): return int(round(y0B+(vyB/vxB)*(x-x0B)))
    return y_top_at,y_bot_at,xs.min(),xs.max()

def vertical_range_robust(mask: np.ndarray, x: int, *, win: int,
                          y_top_at, y_bot_at, rel: float = 0.6):
    H, W = mask.shape
    x0, x1 = max(0, x - win), min(W - 1, x + win)
    y0p, y1p = y_top_at(x), y_bot_at(x)
    y0p = max(0, min(H - 1, y0p))
    y1p = max(0, min(H - 1, y1p))
    col_any = mask[:, x0:x1+1].any(axis=1)
    ys = np.where(col_any)[0]
    if ys.size:
        y0, y1 = int(ys.min()), int(ys.max())
        if (y1 - y0) < rel * max(1, (y1p - y0p)):
            return y0p, y1p
        return y0, y1
    return y0p, y1p

# --- YOLO
model = YOLO(MODEL_PATH)
pred = model.predict(img, conf=CONF, iou=IOU, imgsz=1024,
                     rect=True, save=False, retina_masks=True)[0]

num_masks = 0 if (pred.masks is None or pred.masks.data is None) else len(pred.masks.data)
print(f"🔎 Objets détectés: {num_masks}")
overlay = img.copy()

if num_masks == 0:
    # rien détecté -> sauvegarde l’image corrigée + message clair
    cv2.imwrite(OUT_PATH, overlay)
    print("⚠️  Aucun masque détecté par YOLO sur cette image.")
else:
    for m_raw, box in zip(pred.masks.data.cpu().numpy(),
                          pred.boxes.xyxy.cpu().numpy().astype(int)):

        # 1) masque full-res
        mask = cv2.resize((m_raw>0.35).astype(np.uint8)*255,
                          img.shape[:2][::-1], cv2.INTER_NEAREST)
        mask = clean_mask(mask)

        # 2) droites haut/bas (fallback)
        y_top_at, y_bot_at, xs_min, xs_max = fit_top_bottom_lines(mask)

        # --- Homographie vers plan métrique (H_CM fixe, W variable)
        H_CM      = 201.5
        PX_PER_CM = 10

        xL, xR = xs_min, xs_max
        pTL = (xL, y_top_at(xL));  pTR = (xR, y_top_at(xR))
        pBR = (xR, y_bot_at(xR));  pBL = (xL, y_bot_at(xL))

        h_px_left  = pBL[1] - pTL[1]
        h_px_right = pBR[1] - pTR[1]
        h_px_avg   = (h_px_left + h_px_right)/2.0
        SCALE_Y    = H_CM / max(h_px_avg, 1e-6)

        w_px_top = np.hypot(pTR[0]-pTL[0], pTR[1]-pTL[1])
        w_px_bot = np.hypot(pBR[0]-pBL[0], pBR[1]-pBL[1])
        W_CM_est = SCALE_Y * (w_px_top + w_px_bot) / 2.0

        W_out = int(round(W_CM_est*PX_PER_CM))
        H_out = int(round(H_CM   *PX_PER_CM))

        src = np.float32([pTL,pTR,pBR,pBL])
        dst = np.float32([[0,0],[W_out-1,0],[W_out-1,H_out-1],[0,H_out-1]])
        Hmat, _ = cv2.findHomography(src, dst)

        def to_metric_cm(pts_xy):
            pts_xy = np.asarray(pts_xy, np.float32).reshape(-1,1,2)
            pts_m  = cv2.perspectiveTransform(pts_xy, Hmat).reshape(-1,2)
            return pts_m / PX_PER_CM

        # 3) quadrillage & mesures
        grid = np.zeros((*mask.shape,3), np.uint8)

        # VERTICALES
        for t in np.linspace(0, 1, N_LEN):
            x = int(round(xs_min + t*(xs_max - xs_min)))
            if t == 0 or t == 1:
                y0, y1 = y_top_at(x), y_bot_at(x)
            else:
                y0, y1 = vertical_range_robust(
                    mask, x, win=6, y_top_at=y_top_at, y_bot_at=y_bot_at, rel=0.6
                )
            if y0 is None: 
                continue
            cv2.line(grid, (x,y0), (x,y1), COL_V, THICK_V)
            P0_cm, P1_cm = to_metric_cm([(x,y0),(x,y1)])
            long_cm_v = float(np.linalg.norm(P1_cm - P0_cm))
            cv2.putText(overlay, f"{long_cm_v:.0f} cm",
                        (x + 8, (y0 + y1)//2),
                        FONT, 0.9, TXT_COL, TXT_TH, cv2.LINE_AA)

        # HORIZONTALES (bord haut / bas + une interne)
        ys_mask = np.where(mask.any(axis=1))[0]
        if ys_mask.size:
            y_min, y_max = ys_mask.min(), ys_mask.max()

            x_left, x_right = xs_min, xs_max
            y_left_top  = y_top_at(x_left)
            y_right_top = y_top_at(x_right)
            cv2.line(grid, (x_left,y_left_top), (x_right,y_right_top), COL_H, THICK_H)
            A_cm,B_cm   = to_metric_cm([(x_left,y_left_top), (x_right,y_right_top)])
            long_cm_top = float(np.linalg.norm(B_cm - A_cm))
            cv2.putText(overlay, f"{long_cm_top:.2f} cm",
                        ((x_left + x_right)//2, min(y_left_top, y_right_top) - 10),
                        FONT, 0.9, TXT_COL, TXT_TH, cv2.LINE_AA)

            bottom_pts=[]
            for x in np.where(mask.any(axis=0))[0]:
                ys = np.where(mask[:,x])[0]
                bottom_pts.append([x, ys.max()])
            bottom_pts = np.array(bottom_pts, dtype=np.float32)
            vx,vy,x0b,y0b = cv2.fitLine(bottom_pts, cv2.DIST_L2, 0, 0.01, 0.01)
            x_left, x_right = int(bottom_pts[:,0].min()), int(bottom_pts[:,0].max())
            y_left  = int(y0b + vy/vx * (x_left  - x0b))
            y_right = int(y0b + vy/vx * (x_right - x0b))
            cv2.line(grid, (x_left,y_left), (x_right,y_right), COL_H, THICK_H)
            A2_cm,B2_cm  = to_metric_cm([(x_left,y_left), (x_right,y_right)])
            long_cm_bot  = float(np.linalg.norm(B2_cm - A2_cm))
            cv2.putText(overlay, f"{long_cm_bot:.2f} cm",
                        ((x_left + x_right)//2, y_right + 22),
                        FONT, 0.9, TXT_COL, TXT_TH, cv2.LINE_AA)

            for s in np.linspace(0, 1, N_WID+2)[1:-1]:
                y_mid = int(round(y_min + s*(y_max - y_min)))
                xs_mid = np.where(mask[y_mid])[0]
                if not xs_mid.size: 
                    continue
                cv2.line(grid, (xs_mid.min(), y_mid), (xs_mid.max(), y_mid), COL_H, THICK_H)
                A3_cm,B3_cm = to_metric_cm([(xs_mid.min(), y_mid), (xs_mid.max(), y_mid)])
                long_cm_mid = float(np.linalg.norm(B3_cm - A3_cm))
                cv2.putText(overlay, f"{long_cm_mid:.2f} cm",
                            ((xs_mid.min() + xs_mid.max()) // 2, y_mid + 22),
                            FONT, 0.9, TXT_COL, TXT_TH, cv2.LINE_AA)

        # blend masque + grid
        mask_rgb = cv2.merge([np.zeros_like(mask), mask, np.zeros_like(mask)])
        overlay  = cv2.addWeighted(overlay, 1.0, mask_rgb, MASK_ALPHA, 0)
        overlay  = cv2.addWeighted(overlay, 1.0, grid,      1.0,       0)

# =========================
# 4) SAVE
# =========================
cv2.imwrite(OUT_PATH, overlay)
print(f"✅ Image finale avec mesures → {OUT_PATH}")
cv2.imshow("Résultat", cv2.resize(overlay, (1200, 700)))
cv2.waitKey(0); cv2.destroyAllWindows()


✅ Undistort appliqué
✅ Rotation appliquée

0: 640x1024 1 plaque, 5419.6ms
Speed: 49.0ms preprocess, 5419.6ms inference, 134.2ms postprocess per image at shape (1, 3, 640, 1024)
🔎 Objets détectés: 1


C:\Users\Hp\AppData\Local\Temp\ipykernel_8408\806842886.py:240: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_left  = int(y0b + vy/vx * (x_left  - x0b))
C:\Users\Hp\AppData\Local\Temp\ipykernel_8408\806842886.py:241: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_right = int(y0b + vy/vx * (x_right - x0b))


✅ Image finale avec mesures → D:\Users\Hp\Desktop\Inspection_Plaques\brute04_result.png
